In [32]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix

In [23]:
file_path = "Alzheimer_large_data_dirty.csv"
data = pd.read_csv(file_path)
print(data.head())

    Age  MemoryLossSeverity  CognitiveImpairmentScore FamilyHistory  \
0  79.0            1.880604                 34.431475           NaN   
1  31.0                 NaN                       NaN           Yes   
2  54.0            4.591552                 58.254542           Yes   
3   NaN            2.390679                 73.767240            No   
4  80.0                 NaN                 36.628152           Yes   

   LifestyleScore EducationLevel  PhysicalActivityHours  ChronicIllnessCount  \
0        6.511625            PhD               3.224130                    3   
1        3.603621    High School               3.372636                    0   
2        5.004620         Master               0.712157                    0   
3        2.948366            PhD               4.652270                    0   
4        2.216656         Master               4.072091                    1   

  SmokingStatus                  Diagnosis  
0    Non-Smoker  Mild Cognitive Impairment  
1 

In [24]:
# فحص القيم المفقودة
print("عدد القيم المفقودة في كل عمود:")
print(data.isnull().sum())

# فحص القيم غير الصحيحة
print("\nفحص الأعمدة العددية بحثًا عن القيم غير الصحيحة:")
for column in ["Age", "MemoryLossSeverity", "CognitiveImpairmentScore", "PhysicalActivityHours", "ChronicIllnessCount"]:
    invalid_rows = data[~data[column].astype(str).str.isnumeric()]
    if not invalid_rows.empty:
        print(f"القيم غير الصحيحة في العمود {column}:")
        print(invalid_rows[[column]])

# تنظيف البيانات
# تحويل القيم غير الصحيحة إلى NaN
for column in ["Age", "MemoryLossSeverity", "CognitiveImpairmentScore", "PhysicalActivityHours", "ChronicIllnessCount"]:
    data[column] = pd.to_numeric(data[column], errors="coerce")

# التعامل مع القيم المفقودة:
# استبدال القيم المفقودة بمتوسط العمود (للأعمدة العددية)
numeric_columns = ["Age", "MemoryLossSeverity", "CognitiveImpairmentScore", "PhysicalActivityHours", "ChronicIllnessCount"]
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

# استبدال القيم المفقودة بأكثر القيم تكرارًا (للأعمدة النصية)
categorical_columns = ["EducationLevel", "SmokingStatus"]
for column in categorical_columns:
    data[column] = data[column].fillna(data[column].mode()[0])
    

# عرض البيانات بعد التنظيف
print("\nعرض أول 5 صفوف بعد التنظيف:")
print(data.head())

# حفظ البيانات بعد التنظيف
data.to_csv("Alzheimer_cleaned_data.csv", index=False)
print("\nتم تنظيف البيانات وحفظها في ملف Alzheimer_cleaned_data.csv.")


عدد القيم المفقودة في كل عمود:
Age                         1000
MemoryLossSeverity          1000
CognitiveImpairmentScore    1000
FamilyHistory                500
LifestyleScore                 0
EducationLevel               500
PhysicalActivityHours       1000
ChronicIllnessCount            0
SmokingStatus                500
Diagnosis                      0
dtype: int64

فحص الأعمدة العددية بحثًا عن القيم غير الصحيحة:
القيم غير الصحيحة في العمود Age:
       Age
0     79.0
1     31.0
2     54.0
3      NaN
4     80.0
...    ...
9995  67.0
9996  42.0
9997  46.0
9998  38.0
9999  52.0

[10000 rows x 1 columns]
القيم غير الصحيحة في العمود MemoryLossSeverity:
      MemoryLossSeverity
0               1.880604
1                    NaN
2               4.591552
3               2.390679
4                    NaN
...                  ...
9995            5.081636
9996            9.150874
9997            3.953527
9998            1.318904
9999            5.861402

[10000 rows x 1 columns]
القيم غير ال

In [29]:
print(data.dtypes)

Age                         float64
MemoryLossSeverity          float64
CognitiveImpairmentScore    float64
FamilyHistory                object
LifestyleScore              float64
EducationLevel                 int8
PhysicalActivityHours       float64
ChronicIllnessCount           int64
SmokingStatus                  int8
Diagnosis                    object
dtype: object


In [25]:

print("\nعرض أول 5 صفوف بعد التنظيف:")
print(data.head())

data.to_csv("Alzheimer_cleaned_data.csv", index=False)
print("\nتم تنظيف البيانات وحفظها في ملف Alzheimer_cleaned_data.csv.")


عرض أول 5 صفوف بعد التنظيف:
         Age  MemoryLossSeverity  CognitiveImpairmentScore FamilyHistory  \
0  79.000000            1.880604                 34.431475           NaN   
1  31.000000            4.995463                 49.858909           Yes   
2  54.000000            4.591552                 58.254542           Yes   
3  59.074111            2.390679                 73.767240            No   
4  80.000000            4.995463                 36.628152           Yes   

   LifestyleScore EducationLevel  PhysicalActivityHours  ChronicIllnessCount  \
0        6.511625            PhD               3.224130                    3   
1        3.603621    High School               3.372636                    0   
2        5.004620         Master               0.712157                    0   
3        2.948366            PhD               4.652270                    0   
4        2.216656         Master               4.072091                    1   

  SmokingStatus                  

In [31]:
# تحميل البيانات النظيفة
file_path = "Alzheimer_cleaned_data.csv"
data = pd.read_csv(file_path)

# تحويل الأعمدة النصية إلى قيم رقمية باستخدام الترميز
data["EducationLevel"] = data["EducationLevel"].astype("category").cat.codes
data["SmokingStatus"] = data["SmokingStatus"].astype("category").cat.codes
# تحويل العمود FamilyHistory إلى قيم رقمية
data["FamilyHistory"] = data["FamilyHistory"].astype("category").cat.codes

# تحويل العمود Diagnosis إلى قيم رقمية (مثلاً "Positive" = 1 و "Negative" = 0)
data["Diagnosis"] = data["Diagnosis"].astype("category").cat.codes


# تحديد الميزات (Features) والهدف (Target)
X = data.drop("Diagnosis", axis=1)  # إزالة عمود الهدف
y = data["Diagnosis"]  # الهدف

# تقسيم البيانات إلى تدريب (80%) واختبار (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. Logistic Regression
log_model = LogisticRegression(max_iter=1000, random_state=42)
log_model.fit(X_train, y_train)
log_predictions = log_model.predict(X_test)
log_accuracy = accuracy_score(y_test, log_predictions)

print("Logistic Regression:")
print(f"Accuracy: {log_accuracy:.2f}")
print(classification_report(y_test, log_predictions))

# 2. Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)

print("\nRandom Forest:")
print(f"Accuracy: {rf_accuracy:.2f}")
print(classification_report(y_test, rf_predictions))



Logistic Regression:
Accuracy: 0.33
              precision    recall  f1-score   support

           0       0.33      0.34      0.33       673
           1       0.33      0.35      0.34       666
           2       0.33      0.30      0.31       661

    accuracy                           0.33      2000
   macro avg       0.33      0.33      0.33      2000
weighted avg       0.33      0.33      0.33      2000


Random Forest:
Accuracy: 0.34
              precision    recall  f1-score   support

           0       0.35      0.36      0.36       673
           1       0.33      0.32      0.33       666
           2       0.34      0.34      0.34       661

    accuracy                           0.34      2000
   macro avg       0.34      0.34      0.34      2000
weighted avg       0.34      0.34      0.34      2000



In [33]:
# مصفوفة الالتباس للنموذج اللوجستي
log_cm = confusion_matrix(y_test, log_predictions)
print("Logistic Regression Confusion Matrix:\n", log_cm)

# مصفوفة الالتباس لنموذج Random Forest
rf_cm = confusion_matrix(y_test, rf_predictions)
print("\nRandom Forest Confusion Matrix:\n", rf_cm)

Logistic Regression Confusion Matrix:
 [[230 236 207]
 [233 230 203]
 [239 222 200]]

Random Forest Confusion Matrix:
 [[243 213 217]
 [229 214 223]
 [216 219 226]]


In [34]:
import pickle

# حفظ النموذج باستخدام pickle
with open("logistic_regression_model.pkl", "wb") as f:
    pickle.dump(log_model, f)

with open("random_forest_model.pkl", "wb") as f:
    pickle.dump(rf_model, f)

print("تم حفظ النماذج بنجاح.")

تم حفظ النماذج بنجاح.
